In [42]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd

### Conection

In [43]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'Power{}'.format(time.time())
client = bigtable.Client.from_service_account_json('../unbosque-service-account.json', admin = True)
instance = client.instance(  INSTANCE_ID)

In [44]:
p1_gendata = pd.read_csv("../data/Plant_1_Generation_Data.csv")

### Creating Table plant 1 generation data and family

In [45]:
"""Creating Table plant 1 generation data csv"""
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the Power1649486163.977571 table


In [46]:
print('Creating POWER Column')
power_cfId = 'Power'
power_cf = table.column_family(power_cfId)
power_cf.create()

Creating POWER Column


### Inserting data into table plant 1 generation data 

In [48]:
#Aquí me salió el siguiente error: 'Maximum number of mutations is 100000' por ende recurrí a borrar
#Las otras columnas (AC_POWER, DC_POWER, DAILY YIELD)
dt = dt.utcnow()
rows = []

json = p1_gendata.to_dict(orient='records')

print('Writing TOTAL YIELD')

for power in json:
    
    row_key = 'plant1#{}#{}'.format(power['SOURCE_KEY'],power['DATE_TIME']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(power_cfId, 'total_yield'.encode(),str(power['TOTAL_YIELD'] ),timestamp=dt)
    rows.append(row)

table.mutate_rows(rows)
print('Done')
  


Writing TOTAL YIELD
Done


In [49]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode("utf-8")))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = (
                    " [{}]".format(",".join(cell.labels)) if len(cell.labels) else ""
                )
                print(
                    "\t{}: {} @{}{}".format(
                        col.decode("utf-8"),
                        cell.value.decode("utf-8"),
                        cell.timestamp,
                        labels,
                    )
                )
    print("")

In [50]:
row_key = "plant1#adLQvlD726eNBSB#15-05-2020 14:15"
row = table.read_row(row_key)
print_row(row)

Reading data for plant1#adLQvlD726eNBSB#15-05-2020 14:15:
Column Family Power
	total_yield: 6275988.143 @2022-04-09 06:36:39.584000+00:00
	total_yield: 6275988.143 @2022-04-09 06:36:20.774000+00:00

